In [1]:
# Modules: Email_Parser and PDF_Converter
from email.policy import default
from email.parser import BytesParser
import glob
import pdfkit
import os
from bs4 import BeautifulSoup
import datetime 
from pathlib import Path

# Modules: Registration List and Folder Creator
import os 
import pdfquery
import pandas as pd

# Modules: Folder Creator 
import os
import pandas as pd
import numpy as np
import shutil
import pathlib
import xlwings as xw


In [2]:
# extract_file_url() is a function that takes in a list of .eml files.
# It parses the .eml file, extracts the URL from the .eml file
# and returns a dictionary with the file name as the key and the URL as the value

def extract_file_url(student_file):
    student_dict = dict()

    # Iterate through the directory of .eml files
    for student in student_file:
        
        with open(student, 'rb') as fp:  
            # Parse the email, get the link to the registration form.
            rawMessage = BytesParser(policy=default).parse(fp)
            soup = BeautifulSoup(rawMessage.get_body(preferencelist=('html')).get_content(), 'html.parser')
            registration_link = None
            try:
                registration_link = soup.find_all('a')[1].get('href')
                student_dict[student] = [registration_link]

                # Get the time the email was submitted, format the time. 
                time_submitted = rawMessage['Received'].split(';')[1].strip()
                time_submitted = datetime.datetime.strptime(time_submitted, '%a, %d %b %Y %H:%M:%S %z')
                time_submitted = time_submitted.strftime('%m-%d-%Y %H:%M:%S')
                student_dict[student].append(time_submitted)
            
            except TypeError:
                print(f"No URL found in {student}")
                continue

    return student_dict

name = extract_file_url(glob.glob('*.eml'))


In [3]:
# get_pdf() is a function that takes in a dictionary of student names and their registration form URL.
# It uses pdfkit to convert the URL to a PDF and saves the PDF in the same directory as the script.
# TODO: Remove the .eml files after the PDFs have been generated.

def get_pdf(student_dict):
    
    count = 0
    
    options = {'orientation': 'Landscape'}

    for key, value in student_dict.items():
        out_file = key + '.pdf'
        pdfkit.from_url(value, out_file,options = options)
        count +=1
        
    print('Done!')
    print(f'A total of {count} pdfs were generated.')

get_pdf(name)

Done!
A total of 11 pdfs were generated.


In [4]:
def move_pdf():

    os.mkdir("Generated PDFs")
    for file in glob.glob("*.pdf"):
        os.rename(file, "Generated PDFs/" + file)
    
    print('Done!')
    print('All PDFs have been moved to the Generated PDFs folder.')


move_pdf()


Done!
All PDFs have been moved to the Generated PDFs folder.


At this point the following has been created: 

1) PDFs of Student Applications have been generated and saved locally. 
2) Information related to applications has been scraped loaded into xlsx file for further processing. 
